In [9]:
import os
import pypsa
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

idx = pd.IndexSlice

In [10]:
files = [
    'nodal_capacities.csv',
    'nodal_geothermal_stats.csv',
    'nodal_loads.csv',
    'costs.csv',
    'capacities.csv',
    'supply.csv',
    'supply_energy.csv',
]

In [11]:
base = Path.cwd().parent.parent.parent / 'cluster_data' / 'breyer_sweep'
path1 = base / 'sensitivity_renewable_costs_heat'
path2 = base / 'sensitivity_renewable_costs_elec'
target = base / 'sensitivity_renewable_costs_all'

In [12]:
index_cols = {
    'nodal_capacities.csv': list(range(3)),
    'nodal_geothermal_stats.csv': list(range(2)),
    'nodal_loads.csv': list(range(2)),
    'costs.csv': list(range(3)),
    'capacities.csv': list(range(2)),
    'supply.csv': list(range(3)),
    'supply_energy.csv': list(range(3)),
}

for file in files:
    df1 = pd.read_csv(path1 / file, index_col=index_cols[file], header=list(range(9)))
    df2 = pd.read_csv(path2 / file, index_col=index_cols[file], header=list(range(9)))

    pd.concat([df1, df2], axis=1).to_csv(target / file)

In [68]:
def get_capacity_or_cost(path, mode, name):
    """Obtains base numbers for elec, dh, chp. For now these are installed borehole capacity
    and total system cost."""

    assert mode in ['capacity', 'cost']
    fn = 'capacities.csv' if mode == 'capacity' else 'costs.csv'

    df = pd.read_csv(
        path / fn,
        index_col=[0,1],
        header=list(range(9))
        )
    # print(df.index.get_level_values(1).unique())
    flex = 'static'
    waste_heat = 'True'
    dh_share = '0.3'
    ll = 'v1.0'
    so = 'Co2L0-3H-T-H-B-I-solar+p3-dist1'

    # print(df.iloc[:3,:3])

    if name == 'flex':
        flex = 'flex'
    elif name == 'waste_heat':
        waste_heat = 'False'

    elif 'dh_share' in name:
        dh_share = name.split('-')[-1]
    elif 'trans' in name:
        ll = name.split('-')[-1]
    
    elif 'rn_costs' in name:
        c = name.split('-')[-1]
        template = '-wind+c{}-solar+c{}-ror+c{}-hydro+c{}-nuclear+c{}'
        so += template.format(c,c,c,c,c)

    if mode == 'capacity':
        if name != 'waste_heat' and 'dh_share' not in name:
            elec = df.loc[
                    idx['links', ['injection geothermal heat']]
                    ,
                    idx[:,ll,so,:,'2050',['elec'], flex, dh_share, waste_heat],
                    ]
        else: elec = pd.DataFrame()

        ss = pd.concat((
            df.loc[
                idx['links', ['injection geothermal heat']]
                ,
                idx[:,ll,so,:,'2035',['chp','dh'], flex, dh_share, waste_heat],
                ],
            elec
        )).T.fillna(0).sum(axis=1)
    else:
        if name != 'waste_heat' and 'dh_share' not in name:
            elec = df.loc[
                :,
                idx[:,ll,so,:,'2050', ['elec'], flex, dh_share, waste_heat],
                ]
        else: elec = pd.DataFrame()

        ss = pd.concat((
            df.loc[
                :,
                idx[:,ll,so,:,'2035',['chp','dh'], flex, dh_share, waste_heat],
                ],
            elec
        ), axis=1).sum()

    ss.index = ss.index.get_level_values(5)

    ss = pd.DataFrame(ss).rename(columns={0:name}).stack()
    ss = pd.DataFrame(ss).rename(columns={0:mode}).stack()
    ss.index = ss.index.reorder_levels([2,1,0])

    return ss


pd.concat((
    get_capacity_or_cost(base/'joint_data', 'capacity', 'base'),
    get_capacity_or_cost(base/'joint_data', 'cost', 'base'),

    get_capacity_or_cost(base/'joint_data', 'capacity', 'flex'),
    get_capacity_or_cost(base/'joint_data', 'cost', 'flex'),

    get_capacity_or_cost(base/'sensitivity_transmission_all', 'capacity', 'trans-v1.125'),
    get_capacity_or_cost(base/'sensitivity_transmission_all', 'cost', 'trans-v1.125'),
    get_capacity_or_cost(base/'sensitivity_transmission_all', 'capacity', 'trans-v1.25'),
    get_capacity_or_cost(base/'sensitivity_transmission_all', 'cost', 'trans-v1.25'),

    get_capacity_or_cost(base/'sensitivity_waste_heat', 'capacity', 'waste_heat'),
    get_capacity_or_cost(base/'sensitivity_waste_heat', 'cost', 'waste_heat'),

    get_capacity_or_cost(base/'sensitivity_renewable_costs_all', 'capacity', 'rn_costs-0.8'),
    get_capacity_or_cost(base/'sensitivity_renewable_costs_all', 'cost', 'rn_costs-0.8'),
    get_capacity_or_cost(base/'sensitivity_renewable_costs_all', 'capacity', 'rn_costs-1.2'),
    get_capacity_or_cost(base/'sensitivity_renewable_costs_all', 'cost', 'rn_costs-1.2'),

    get_capacity_or_cost(base/'sensitivity_district_heat', 'capacity', 'dh_share-0.0'),
    get_capacity_or_cost(base/'sensitivity_district_heat', 'cost', 'dh_share-0.0'),
    get_capacity_or_cost(base/'sensitivity_district_heat', 'capacity', 'dh_share-0.6'),
    get_capacity_or_cost(base/'sensitivity_district_heat', 'cost', 'dh_share-0.6'),
    get_capacity_or_cost(base/'sensitivity_district_heat', 'capacity', 'dh_share-1.0'),
    get_capacity_or_cost(base/'sensitivity_district_heat', 'cost', 'dh_share-1.0'),
))

                        egs_mode
capacity  base          chp         1.844976e+05
                        dh          1.250311e+05
                        elec        3.331386e+06
cost      base          chp         8.388875e+11
                        dh          8.404916e+11
                        elec        8.055913e+11
capacity  flex          chp         1.748220e+05
                        dh          1.131417e+05
                        elec        3.614839e+06
cost      flex          chp         8.368314e+11
                        dh          8.387539e+11
                        elec        7.933374e+11
capacity  trans-v1.125  chp         1.776887e+05
                        dh          1.246473e+05
                        elec        3.342512e+06
cost      trans-v1.125  chp         8.199522e+11
                        dh          8.213615e+11
                        elec        7.895003e+11
capacity  trans-v1.25   chp         1.736943e+05
                        dh          

In [14]:
n1 = pypsa.Network('test/' + os.listdir('test')[0])
n2 = pypsa.Network('test/' + os.listdir('test')[1])

INFO:pypsa.io:Imported network elec_s_72_lv1.0__Co2L0-180H-T-H-B-I-solar+p3-dist1-onwind+c0.8-offwind+c0.8-solar+c0.8_2050_2035_chp_static_0.3_True.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Imported network elec_s_72_lv1.0__Co2L0-180H-T-H-B-I-solar+p3-dist1_2050_2035_chp_static_0.3_True.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


In [15]:
pd.concat((
    n1.generators.groupby('carrier')['capital_cost'].mean(),
    n2.generators.groupby('carrier')['capital_cost'].mean()), axis=1)

,capital_cost,capital_cost
carrier,,
gas,0.000000,0.000000
geothermal heat,0.000000,0.000000
offwind-ac,150823.258290,188529.072862
offwind-dc,169312.627909,211640.784886
oil,0.000000,0.000000
onwind,90169.989858,109387.167462
residential rural solar thermal,18701.658071,23377.072589
residential urban decentral solar thermal,18701.658071,23377.072589
ror,299140.224929,299140.224929


In [22]:
(g := pd.concat((
    n1.links.groupby('carrier')['capital_cost'].mean(),
    n2.links.groupby('carrier')['capital_cost'].mean()), axis=1
    )).loc[g.index.str.contains('C')]

,capital_cost,capital_cost
carrier,,
DAC,863357.554460,863357.554460
DC,38612.553511,38612.553511
H2 Fuel Cell,105807.626500,105807.626500
OCGT,18489.131021,18489.131021
SMR CC,74751.008217,74751.008217
gas for industry CC,60221.468955,60221.468955
process emissions CC,301107.344774,301107.344774
solid biomass for industry CC,111409.717566,111409.717566
urban central gas CHP,27324.814754,27324.814754
